## 图像轮廓

图像轮廓可以简单认为成将连续的点（连着边界）连在一起的曲线，具有相同的颜色或者灰度。轮廓在形状分析和物体的检测和识别中很有用。

- 为了更加准确，要使用二值化图像。在寻找轮廓之前，要进行阈值化处理，或者Canny边界检测。
- 查找轮廓的函数会修改原始图像。如果你在找到轮廓之后还想使用原始图像的话，你应该将原始图像存储到其他变量中。
- 在OpenCV中，查找轮廓就像在黑色背景中超白色物体。你应该记住要找的物体应该是白色而背景应该是黑色。

### cv2.findContours()函数

```python
cv2.findContours(image, mode, method[, contours[, hierarchy[, offset ]]])  
```
注意：opencv2返回两个值：contours：hierarchy。而opencv3会返回三个值,分别是img（图像）, countours（轮廓，是一个列表，里面存贮着图像中所有的轮廓，每一个轮廓都是一个numpy数组，包含对象边界点（x, y）的坐标）, hierarchy（轮廓的层析结构）

函数参数：
- 第一个参数是寻找轮廓的图像，即输入图像；
- 第二个参数表示轮廓的检索模式，有四种（本文介绍的都是新的cv2接口）：
    1. cv2.RETR_EXTERNAL： 表示只检测外轮廓
    2. cv2.RETR_LIST： 表示检测所有轮廓，检测的轮廓不建立等级关系，并将其保存到一条链表当中
    3. cv2.RETR_CCOMP ：表示检测所有的轮廓，并将他们组织为两层：顶层是各部分的外部边界，第二次是空洞的边界
    4. cv2.RETR_TREE： 表示检测所有轮廓，并重构嵌套轮廓的整个层次，建立一个等级树结构的轮廓
- 第三个参数method为轮廓的近似办法
    1. cv2.CHAIN_APPROX_NONE：以Freeman链码的方式输出轮廓，所有其他方法输出多边形（顶点的序列）。存储所有的轮廓点，相邻的两个点的像素位置差不超过1，即max（abs（x1-x2），abs（y2-y1））==1
    2. cv2.CHAIN_APPROX_SIMPLE：压缩水平方向，垂直方向，对角线方向的元素，只保留该方向的终点坐标，例如一个矩形轮廓只需4个点来保存轮廓信息
    3. cv2.CHAIN_APPROX_TC89_L1，CV_CHAIN_APPROX_TC89_KCOS使用teh-Chinl chain 近似算法

函数返回值：
1. contour返回值：cv2.findContours()函数首先返回一个 list，list中每个元素都是图像中的一个轮廓，用numpy中的ndarray表示。
2. hiarachy返回值：这是一个ndarray，其中的元素个数和轮廓个数相同，每个轮廓contours[i]对应4个hierarchy元素hierarchy[i][0] ~hierarchy[i][3]，分别表示后一个轮廓、前一个轮廓、父轮廓、内嵌轮廓的索引编号，如果没有对应项，则该值为负数。

### cv2.drawContours()

```python
cv2.drawContours(image, contours, contourIdx, color[,thickness[, lineType[, hierarchy[, maxLevel[, offset ]]]]])
```

参数：

- 第一个参数是指明在哪幅图像上绘制轮廓；
- 第二个参数是轮廓本身，在Python中是一个list。
- 第三个参数指定绘制轮廓list中的哪条轮廓，如果是-1，则绘制其中的所有轮廓。后面的参数很简单。其中thickness表明轮廓线的宽度，如果是-1（cv2.FILLED），则为填充模式。绘制参数将在以后独立详细介绍。


### cv2.boundingrect()函数

```python
def boundingRect(array): # real signature unknown; restored from __doc__
    """
    boundingRect(array) -> retval
    .   @brief Calculates the up-right bounding rectangle of a point set or non-zero pixels of gray-scale image.
    .  
    .   The function calculates and returns the minimal up-right bounding rectangle for the specified point set or
    .   non-zero pixels of gray-scale image.
    .  
    .   @param array Input gray-scale image or 2D point set, stored in std::vector or Mat.
    """
    pass
```

In [1]:
import cv2
import numpy as np


img_path = '../image/ImageContour/contour2.png'
img = cv2.imread(img_path)
img1 = img.copy()
img2 = img.copy()
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
contours, hierarchy= cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
print('轮廓的总数为', len(contours))
# 轮廓的总数为 2
 
cnt = contours[0]
x, y, w, h = cv2.boundingRect(cnt)
img1 = cv2.rectangle(img1, (x,y), (x+w,y+h), (0, 255, 0), 2)
 
cv2.imshow('img', img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

轮廓的总数为 2


### cv2.contourArea()

```python
def contourArea(contour, oriented=None): # real signature unknown; restored from __doc__
    """
    contourArea(contour[, oriented]) -> retval
    .   @brief Calculates a contour area.
    .  
    .   The function computes a contour area. Similarly to moments , the area is computed using the Green
    .   formula. Thus, the returned area and the number of non-zero pixels, if you draw the contour using
    .   #drawContours or #fillPoly , can be different. Also, the function will most certainly give a wrong
    .   results for contours with self-intersections.
    .  
    .   Example:
    .   @code
    .       vector<Point> contour;
    .       contour.push_back(Point2f(0, 0));
    .       contour.push_back(Point2f(10, 0));
    .       contour.push_back(Point2f(10, 10));
    .       contour.push_back(Point2f(5, 4));
    .  
    .       double area0 = contourArea(contour);
    .       vector<Point> approx;
    .       approxPolyDP(contour, approx, 5, true);
    .       double area1 = contourArea(approx);
    .  
    .       cout << "area0 =" << area0 << endl <<
    .               "area1 =" << area1 << endl <<
    .               "approx poly vertices" << approx.size() << endl;
    .   @endcode
    .   @param contour Input vector of 2D points (contour vertices), stored in std::vector or Mat.
    .   @param oriented Oriented area flag. If it is true, the function returns a signed area value,
    .   depending on the contour orientation (clockwise or counter-clockwise). Using this feature you can
    .   determine orientation of a contour by taking the sign of an area. By default, the parameter is
    .   false, which means that the absolute value is returned.
    """
    pass
```

参数含义如下：

- contour：表示某输入单个轮廓，为array
- oriented：表示某个方向上轮廓的面积值，这里指顺时针或者逆时针。若为True，该函数返回一个带符号的面积值，正负值取决于轮廓的方向（顺时针还是逆时针），若为False，表示以绝对值返回- 

In [2]:
img = cv2.imread(img_path)
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
contours, hierarchy= cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
  
  
cnt = contours[0]
# 求轮廓的面积
area = cv2.contourArea(cnt)
print(img.shape)  # (306, 453, 3)
print(area)  # 57436.5
# 也可以看轮廓面积与边界矩形比
x, y, w, h = cv2.boundingRect(cnt)
rect_area = w*h
extent = float(area) / rect_area
print('轮廓面积与边界矩形比为', extent)

(306, 453, 3)
57436.5
轮廓面积与边界矩形比为 0.7800798598378357


### cv2.arcLength()

```python
def arcLength(curve, closed): # real signature unknown; restored from __doc__
    """
    arcLength(curve, closed) -> retval
    .   @brief Calculates a contour perimeter or a curve length.
    .  
    .   The function computes a curve length or a closed contour perimeter.
    .  
    .   @param curve Input vector of 2D points, stored in std::vector or Mat.
    .   @param closed Flag indicating whether the curve is closed or not.
    """
    pass
```
参数含义如下：

- curve：输入的二维点集（轮廓顶点），可以是 vector或者Mat类型
- closed：用于指示曲线是否封闭

In [3]:
img = cv2.imread(img_path)
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
contours, hierarchy= cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
 
 
cnt = contours[0]
# 求轮廓的周长
arcLength = cv2.arcLength(cnt, True)
print(img.shape)  # (306, 453, 3)
print(arcLength)  # 1265.9625457525253

(306, 453, 3)
1265.9625457525253


### cv2.approxPolyDP()

其参数含义：
- curve：表示输入的点集
- epslion：指定的精度，也即原始曲线与近似曲线之间的最大距离，不过这个值我们一般按照周长的大小进行比较
- close：若为True，则说明近似曲线为闭合的；反之，若为False，则断开

该函数采用的是道格拉斯—普克算法（Douglas-Peucker）来实现。该算法也以Douglas-Peucker 算法和迭代终点拟合算法为名。是将曲线近似表示为一系列点，并减少点的数量的一种算法。该算法的原始类型分别由乌尔斯-拉默（Urs Ramer）于 1972年以及大卫-道格拉斯（David Douglas）和托马斯普克（Thomas Peucker）于 1973年提出，并在之后的数十年中由其他学者完善。

In [4]:
img = cv2.imread(img_path)
img1 = img.copy()
img2 = img.copy()
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
contours, hierarchy= cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
 
cnt = contours[0]
# 绘制独立轮廓，如第四个轮廓
img1 = cv2.drawContours(img1, [cnt], -1, (0, 255, 0), 3)
 
epsilon = 0.1*cv2.arcLength(cnt, True)
approx = cv2.approxPolyDP(cnt, epsilon, True)
img2 = cv2.drawContours(img2, [approx], -1, (0, 255, 0), 3)
 
res = np.hstack((img, img1, img2))
cv2.imshow('img', res)
cv2.waitKey(0)
cv2.destroyAllWindows()

### cv2.minEnclosingCircle()

In [5]:
img = cv2.imread(img_path)
img1 = img.copy()
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY)
contours, hierarchy= cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
  
  
cnt = contours[0]
# 求轮廓的外接圆
(x, y), radius = cv2.minEnclosingCircle(cnt)
center = (int(x), int(y))
radius = int(radius)
img1 = cv2.circle(img1, center, radius, (0, 255, 0), 2)
res = np.hstack((img, img1))
cv2.imshow('img', res)
cv2.waitKey(0)
cv2.destroyAllWindows()